# Benchmark 02

- `OneHotEncoder` Prise en compte des seuls termes ayant généré une fraude
- `RandomForestClassifier`


In [4]:
from utils import *
x_train, y_train, df, x_test = get_data()

# Listes des occurences uniques de valeurs pour les différents types de colonnes
# Prise en compte des seuls termes ayant généré une fraude
df_fraud = df.query('fraud==1')
categories = distinct_values_for('category_', df_fraud)
manufacturers = distinct_values_for('manufacturer_', df_fraud)
products = distinct_values_for('product_', df_fraud)
product_codes = distinct_values_for('product_code_', df_fraud)


In [5]:
# En phase d'évaluation des performances
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train,
                                                   y_train, test_size=0.2, random_state=42)


# Taux de fraude dans l'ensemble d'entraînement
fraud_rate_in_train =  sum(y_train.fraud)/len(y_train.fraud)*100
# Taux de fraude dans l'ensemble de test
fraud_rate_in_test = sum(y_test.fraud)/len(y_test.fraud)*100

# On s'assure qu'il y a moins de 10% d'écart entre les taux de fraude
assert abs(fraud_rate_in_train-fraud_rate_in_test)/fraud_rate_in_train < 0.1

In [6]:
# Colonnes non textuelles supprimées dans la sélection des colonnes pour le 1_hot_encoding
# id, product_count_i, product_type_count
columns_for_1_hot_encoding = ['category_'+str(i+1) for i in range(24)] \
                             + ['manufacturer_'+str(i+1) for i in range(24)] \
                             + ['product_'+str(i+1) for i in range(24)] \
                             + ['product_code_'+str(i+1) for i in range(24)]

x = x_train[columns_for_1_hot_encoding].to_numpy()

In [7]:
from sklearn.preprocessing import OneHotEncoder
unique_values = [categories for i in range(24)]  \
                + [manufacturers for i in range(24)]  \
                + [products for i in range(24)] \
                + [product_codes for i in range(24)] 
                
encoder = OneHotEncoder(categories=unique_values, handle_unknown='ignore')

encoder.fit(x)

x_dummy = encoder.transform(x)
y = y_train.to_numpy()

In [13]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=250)
model.fit(x_dummy, y.ravel())


RandomForestClassifier(n_estimators=250)

In [14]:
# Evaluation 
x_test_dummy = encoder.transform(x_test[columns_for_1_hot_encoding].to_numpy())

y_score = model.predict_proba(x_test_dummy)[:, 1]

from sklearn.metrics import average_precision_score
average_precision_score(y_test.to_numpy(), y_score)

0.14762240983646155

# Proposition de solution

## Entraînement

In [16]:
from utils import *
x_train, y_train, df, x_test = get_data()

# Listes des occurences uniques de valeurs pour les différents types de colonnes
# Prise en compte des seuls termes ayant généré une fraude
df_fraud = df.query('fraud==1')
categories = distinct_values_for('category_', df_fraud)
manufacturers = distinct_values_for('manufacturer_', df_fraud)
products = distinct_values_for('product_', df_fraud)
product_codes = distinct_values_for('product_code_', df_fraud)

# Colonnes non textuelles supprimées dans la sélection des colonnes pour le 1_hot_encoding
# id, product_count_i, product_type_count
columns_for_1_hot_encoding = ['category_'+str(i+1) for i in range(24)] \
                             + ['manufacturer_'+str(i+1) for i in range(24)] \
                             + ['product_'+str(i+1) for i in range(24)] \
                             + ['product_code_'+str(i+1) for i in range(24)]

x = x_train[columns_for_1_hot_encoding].to_numpy()

unique_values = [categories for i in range(24)]  \
                + [manufacturers for i in range(24)]  \
                + [products for i in range(24)] \
                + [product_codes for i in range(24)] 
                
encoder = OneHotEncoder(categories=unique_values, handle_unknown='ignore')
encoder.fit(x)
x_dummy = encoder.transform(x)
y = y_train.to_numpy()

# Entrainement
model = RandomForestClassifier(n_estimators=250)
model.fit(x_dummy, y.ravel())

RandomForestClassifier(n_estimators=250)

## Prédictions

In [17]:
# Evaluation 
x_test_light = x_test[columns_for_1_hot_encoding].to_numpy()
x_dummy = encoder.transform(x_test_light)
y_score = model.predict_proba(x_dummy)[:, 1]

In [18]:
create_submission_csv(y_score, x_test.id, 'y_benchmark_02')

Votre score pour cette soumission est : 0,14272306882230337

Equivalent au benchmark avec n_estimators plus grand dans RandomForestClassifier.